In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.0/605.0 kB 6.1 MB/s eta 0:00:00


In [3]:
from ultralytics import FastSAM
from ultralytics.models.fastsam import FastSAMPrompt
import time
# Define image path and inference device
DEVICE =  "cuda"
# Create a FastSAM model
model = FastSAM('FastSAM-x.pt')  # or FastSAM-x.pt

100%|██████████| 138M/138M [00:00<00:00, 151MB/s]


In [4]:
import cv2
import glob
from google.colab.patches import cv2_imshow
def draw_bounding_box(image, bbox, color=(0, 255, 0), thickness=1):
    bbox = [int(i) for i in bbox]
    x, y, w, h = bbox
    cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)

In [6]:
for fi in glob.glob('/content/gdrive/MyDrive/scene/*'):
  IMAGE_PATH = fi
  image = cv2.imread(IMAGE_PATH)
  st = time.time()
  everything_results = model(IMAGE_PATH,
                            device=DEVICE,
                            retina_masks=True,
                            imgsz=1024,
                            conf=0.5,
                            iou=0.9)
  bboxes_gpu = everything_results[0].boxes.data
  bboxes = bboxes_gpu.cpu().tolist()
  final_boxes = []
  for i in bboxes:
    final_boxes.append(i[:4])
    draw_bounding_box(image, i[:4])
  cv2_imshow(image)
  print(final_boxes)
  prompt_process = FastSAMPrompt(IMAGE_PATH, everything_results, device=DEVICE)
  # # Everything prompt
  ann = prompt_process.everything_prompt()

  prompt_process.plot(annotations=ann, output='output')
  et = time.time()
  # get the execution time
  elapsed_time = et - st
  print('Execution time:', elapsed_time, 'seconds')
  print("____________________")

Output hidden; open in https://colab.research.google.com to view.